# 列出所有的報告

In [2]:
import pandas as pd

In [3]:
hm_NASDAQ_divided = pd.read_csv('../output_dataset/handmade_features_NASDAQ_divided.csv')
hm_NYSE_divided = pd.read_csv('../output_dataset/handmade_features_NYSE_divided.csv')

In [6]:
report_all = pd.concat([hm_NASDAQ_divided, hm_NYSE_divided], axis=0)
report_all = report_all[["folder_name", "exchange", "ticker", "year1",	"year2"]]
report_all.sort_values(by=["folder_name", "exchange", "ticker", "year1", "year2"], inplace=True)

In [7]:
report_all.to_csv('../output_dataset/report_all.csv', index=False)

# 算出每一年的報告數量

In [8]:
import pandas as pd

In [9]:
report_all = pd.read_csv('../output_dataset/report_all.csv')

In [17]:
# Calculate the number of report in each year
report_all['year1'] = report_all['year1'].astype(str)
groupby_year = report_all.groupby(['year1']).size().reset_index(name='count')
groupby_year_wide = groupby_year.set_index('year1').T
groupby_year_wide.to_csv('../output_dataset/groupby_year.csv', index=False)

# 算出每一家公司的 GW score 統計量

In [57]:
import pandas as pd
import numpy as np

In [58]:
report_all = pd.read_csv('../output_dataset/report_all.csv')
common_score_ESG_all = pd.read_csv('../CSR_score/common_score_ESG_all.csv')

In [59]:
# drop those rows that ticker in common_score_ESG_all not in report_all
common_ticker = set(report_all['ticker']) & set(common_score_ESG_all['ticker'])
common_score_ESG_with_report = common_score_ESG_all[common_score_ESG_all['ticker'].isin(common_ticker)]
report_with_score = report_all[report_all['ticker'].isin(common_ticker)]

In [65]:
# find the first year of each ticker
ticker_start = report_with_score['ticker'].unique()
ticker_start = pd.DataFrame(ticker_start, columns=['ticker'])
ticker_start['year1'] = report_with_score.groupby('ticker')['year1'].min().values
ticker_start.to_csv('../output_dataset/ticker_start.csv', index=False)

In [61]:
common_score_ESG_with_report = common_score_ESG_with_report.replace(0, np.nan)
common_score_ESG_with_report = common_score_ESG_with_report.set_index('ticker')

ticker_start = ticker_start.set_index('ticker')
ticker_start['year1'] = ticker_start['year1'].astype(str)

In [64]:
std_result = {}

for ticker in ticker_start.index:
    start_year = ticker_start.loc[ticker, 'year1']

    all_years = common_score_ESG_with_report.columns
    valid_years = [y for y in all_years if y >= start_year]
    scores = common_score_ESG_with_report.loc[ticker, valid_years].dropna()
    std = scores.std()
    std_result[ticker] = std

std_result = pd.DataFrame.from_dict(std_result, orient='index', columns=['std']).reset_index()
std_result.columns = ['ticker', 'std']
std_result.to_csv('../output_dataset/std_result.csv', index=False)

有一些企業沒有std，是因為有報告的起始年份開始沒有分數